In [ ]:
import pandas as pd

In [ ]:
# creating data and dataFrame
experiments_data = {
    'experiment_id': [1, 1, 1],
    'experiment_name': ['exp1', 'exp1', 'exp1'],
    'property_name': ['concentration_unit', 'channel', 'experiment_type'],
    'property_value': ['ul', '1', 'type_a']
}
plates_data = {
    'plate_id': [1, 1, 2, 2],
    'experiment_id': [1, 1, 1, 1],
    'plate_name': ['plate1', 'plate1', 'plate2', 'plate2'],
    'property_name': ['concentration', 'channel', 'concentration', 'channel'],
    'property_value': ['5', '1', '10', '2']
}
wells_data = {
    'well_id': [1, 2, 3, 4, 5, 6],
    'plate_id': [1, 1, 1, 1, 2, 2],
    'well_row': [1, 1, 1, 1, 2, 2],
    'well_column': [1, 2, 3, 4, 1, 2],
    'property_name': ['concentration', 'concentration', 'concentration', 'type', 'type', 'channel'],
    'property_value': ['1', '2', '3', 'A', 'B', '2']
}

In [ ]:
experiments_df = pd.DataFrame(experiments_data)
plates_df = pd.DataFrame(plates_data)
wells_df = pd.DataFrame(wells_data)

In [ ]:
# Pivot the DataFrames
experiments_pivot = experiments_df.pivot_table(index='experiment_id', columns='property_name', values='property_value', aggfunc='first').reset_index()
plates_pivot = plates_df.pivot_table(index=['plate_id', 'experiment_id'], columns='property_name', values='property_value', aggfunc='first').reset_index()
wells_pivot = wells_df.pivot_table(index=['well_id', 'plate_id', 'well_row', 'well_column'], columns='property_name', values='property_value', aggfunc='first').reset_index()

In [ ]:
# Merging
merged_wells_plates = wells_pivot.merge(plates_pivot, on='plate_id', suffixes=('', '_plate'), how='left')
final_df = merged_wells_plates.merge(experiments_pivot, on='experiment_id', suffixes=('', '_experiment'), how='left')

In [ ]:
# Filling missing values with data from higher levels (plate and experiment), keeping the lower level data when available
for col in wells_pivot.columns:
    if col not in ['well_id', 'plate_id', 'well_row', 'well_column']:
        plate_col = col + '_plate'
        experiment_col = col + '_experiment'

        if plate_col in final_df.columns and experiment_col in final_df.columns:
            final_df[col] = final_df[col].combine_first(final_df[plate_col]).combine_first(final_df[experiment_col])
            final_df.drop([plate_col, experiment_col], axis=1, inplace=True)
        elif plate_col in final_df.columns:
            final_df[col] = final_df[col].combine_first(final_df[plate_col])
            final_df.drop([plate_col], axis=1, inplace=True)
        elif experiment_col in final_df.columns:
            final_df[col] = final_df[col].combine_first(final_df[experiment_col])
            final_df.drop([experiment_col], axis=1, inplace=True)

In [ ]:
# Saving results as excel
final_df.to_excel('well_properties.xlsx', index=False)